# Hackathon Bootcamp - Data wrangling workflow

First we'll import some libraries, if you get a `ModuleNotFoundError`, you can simply install it from the Terminal in MacOS and Linux or the Anaconda prompt in windows, using the command: `conda install <module>` or if that fails `pip install <module>`.

So what is **Pandas**? 
It stands for _"Panelled Data Sets”_, so think spreadsheets, BIG spreadsheets.

It's worth noting before we jump in that these amazing libraries (Pandas, NumPy, SciPy, etc) are written by folk who are alive today and who have been amazingly generous with their time. Check out [Wes McKinney's Twitter account](https://twitter.com/wesmckinn) for example, the creator of Pandas. Pandas has got over 10Mio downloads (near 12Mio??).

If you've heard of TensorFlow, it's a similar story: Google opened it up and doubled the number of developers working on it to make it better.

The main point here is that FOSS is simply amazing, and you're all on the brink of contributing to it yourselves: you're in good company!

Be sure to use the [Pandas documentation](https://pandas.pydata.org/pandas-docs/stable/index.html) if you get stuck. Another great resource that [stackoverflow](https://stackoverflow.com) often points me to is [Chris Albon's website](https://chrisalbon.com/)

In [ ]:
import pandas as pd

Now to load some data, we'll use pandas `read_csv()` function, you can use the help methods you learned in `Bootcamp_beginner_track` to see how to use it.

In [ ]:
df = pd.read_csv('../data/training_DataFrame_processed.csv')

With our data loaded, let's start by exploring it.

### Explore the dataset with pandas functions

In [ ]:
df.head()

In [ ]:
df.tail()

In [ ]:
df.columns

In [ ]:
df.info()

In [ ]:
df.nunique()

In [ ]:
df.describe()

### Cleaning Dataframe, dropping and creating columns

In `df.head()` and `df.nunique()` we saw that there is an 'Unnamed: 0' column that we can use as a UID, we will set the index of the Dataframe using this column instead of the automatically generated column, we'll then assign the name of the index to 'UID'. Note that we can also change the name of the colum first and then set the index.

There are no `NaN` values in this dataset, as can be seen from `df.info()` and `df.describe()`, so we will not need to `dropna()`.

Likewise we can check and see there are no duplicates and so we will not need `drop_duplicates()` either on this dataset.

`dropna()` and `drop_duplicates()` are two functions that you may use often, but you must always chose carefuly _what_ you do with `NaN` values and duplicated rows.

In [ ]:
df[df.duplicated(subset='Unnamed: 0') == True].count()

In [ ]:
# SHORTER
# DO ONLY ONE OF THESE TWO OPTIONS: set the index and then change the name of the index
df.set_index('Unnamed: 0', drop=True, inplace=True, verify_integrity=True)
df.index.names = ['UID']

In [ ]:
# EASIER TO READ BUT USES DICT
# DO ONLY ONE OF THESE TWO OPTIONS: change the column name and then reset the index
df.rename(columns={'Unnamed: 0': 'UID'}, inplace=True)
df.set_index('UID', drop=True, inplace=True, verify_integrity=True)

In [ ]:
df.head()

We are now going to drop the 'NM_M' column as we will not use it further, and create a new column called 'Depth_ft' for use by non-SI contractors for example.

In [ ]:
df.drop(columns=['NM_M'], inplace=True)

In [ ]:
df['Depth_ft'] = df['Depth'] * 3.28084

In [ ]:
df.head()

### Investigate data visually

We will first import [seaborn](https://seaborn.pydata.org/), a statistical data visualization package in Python that will allow us to very rapidly look into our data.

In [ ]:
import seaborn as sns

In [ ]:
df.columns

In [ ]:
sns.pairplot(df, vars=['Depth', 'GR', 'ILD_log10',
                       'DeltaPHI', 'PHIND', 'PE', 
                       'RELPOS', 'ILD', 'RHOB'])

The Dataframe we used above is actually already pretty clean, as we can see from `sns.pairplot()`. So now in order to explore Pandas a little more, we will load up a new data set that has not been cleaned yet. The data are interpreted image data showing different data in an inclined borehole.

#### Load and explore the data

We'll start by loading the data, and will then manipulate the resulting Dataframe to illustrate some of Pandas functions.

In [ ]:
df_img = pd.read_csv('../data/image_data.csv')

In [ ]:
df_img.head()

In [ ]:
df_img.info()

Check for any duplicates as the UID column must be... unique.

In [ ]:
df_img[df_img.duplicated(subset='Unnamed: 0') == True].count()

Note that in this data set, it's not uncommon to see duplicate Depths, which we can see using the same method as to check the 'Unnamed: 0' column:

In [ ]:
df_img[df_img.duplicated(subset='#DEPTH') == True]

We'll come back to slicing and indexing, but this should look familiar and allows us to see the duplicated depths easily.

In [ ]:
df_img.iloc[350:352]

Now rename 'Unnamed: 0' to 'UID' and then use that as the index column, as we did before.

In [ ]:
df_img.rename(columns={'Unnamed: 0': 'UID'}, inplace=True)
df_img.set_index('UID', drop=True, inplace=True, verify_integrity=True)

Check that all changes have been carried out successfully:

In [ ]:
df_img.head()

### Slicing and indexing into Dataframe

We can index and slice into Dataframes in many ways, for example:

- slicing, indexing and stepping into columns as we did for lists
- `df.iloc[]` for "Purely integer-location based indexing for selection by position." (see [docs](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.DataFrame.iloc.html))
- `df.loc[]` to "Access a group of rows and columns by label(s) or a boolean array." (see [docs](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.DataFrame.loc.html))

Be sure to see the [docs](https://pandas.pydata.org/pandas-docs/stable/indexing.html) for more details.

In [ ]:
df_img.head()

In [ ]:
df_img[['DIP']][0:3] # pass a list of columns to get nice output

In [ ]:
df_img[['DIP', 'AZIM']][0:10:2]

In [ ]:
df_img.iloc[-20::2]

In [ ]:
df_img.loc[0:5] # note the fact this is NOT half open, contrary to list indexing

### Remove bad or missing data

In this data set there are no 'bad or missing' data _per se_ but there _are_ some data points which we won't look at, so let's start by learning how to drop those rows (you've already seen `pd.drop()` for columns above when we dropped 'NM_M' from the `training_DataFrame_processed.csv` Dataframe.

Here we'll drop both 'BED' and 'BED_LOW_CONF' from the `df_img` Dataframe.

In [ ]:
df_img[df_img.DIPTYPE == 'BED'].index

In [ ]:
df_img.drop(df_img[(df_img.DIPTYPE == 'BED') | (df_img.DIPTYPE == 'BED_LOW_CONF')].index, inplace=True)

Now as always, let's check we made the correct changes:

In [ ]:
df_img.head()

But as you see, we need to correct the 'UID' column _if_ we want it to run sequentially. Note that in Pandas, the index column _does not need_ to be a sequence starting at 0, they don't even need to be unique in fact.

It is good practise as we've been doing so far to keep the index label unique, such as in a time series, if nothing else it makes for easier reading. There is also a [performance hit](https://stackoverflow.com/questions/16626058/what-is-the-performance-impact-of-non-unique-indexes-in-pandas) when using non-unique indices.

This it will also make indexing easier as when we use `pd.iloc[]` the indices will match.

So let's now reset the index from 0.

In [ ]:
df_img.head()

In [ ]:
df_img.reset_index(drop=True, inplace=True)
df_img.index.rename('UID', inplace=True)

In [ ]:
df_img.head()

In [ ]:
df_img.tail()

### Add columns

### Exercise

We saw how to add a column above, try to add a column called '#DEPTH_ft'.

In [ ]:
df_img['#DEPTH_ft'] = df_img['#DEPTH'] * 3.28084

In [ ]:
df_img.head()

### Conditionals

We often want to add new column values based on conditions, we can select rows based on a conditional like we did when we were looking at `duplicated` rows above, following this pattern:

In [ ]:
# Let's select all 'FRACTURE' rows:
df_img[df_img['DIPTYPE'] == 'FRACTURE'].head()

#### Conditionals with numpy

We can also use another library - which we will not explore today - to acess and modify rows based on a condition, with `numpy.where()` which you can read about [here](https://docs.scipy.org/doc/numpy-1.15.1/reference/generated/numpy.where.html).

In [ ]:
import numpy as np

In [ ]:
x = np.arange(100)
y = np.sin(x)

In [ ]:
peaks = y[y>0]
troughs = y[y<0]

In [ ]:
# or we can do:
cond = y > 0
result = y[cond]

And with `numpy` we can use `np.where()` too:

In [ ]:
np.where(df_img['DIP'] >= 75, 'steep', 'shallow')[:10]

### Exercise

With help from the cells above, try to add a new `df_img['ORIGIN']` column with values as follows, using `np.where()`:
- `NATURAL` for natural fractures and,
- `DRILLING` for others

You can find out how many classes of fractures exist by using `df_img.DIPTYPE.unique()`.

N.B.: build this up one step at a time, and remember that multiple conditions will need parentheses `()` surrounding each condition. You will need the boolean operator `|` if you want to represent an `or` case and the `&` if you want to represent an `and` case.

In [ ]:
df_img.DIPTYPE.unique()

In [ ]:
df_img['ORIGIN'] = np.where((df_img['DIPTYPE'] == 'FRACTURE') | (df_img['DIPTYPE'] == 'HEALEDFRACTURE'), 'NATURAL', 'DRILLING')

In [ ]:
df_img.head(10)

### Groupby

Now that we have a new column with natural fractures as well as drilling related damage, we might want to group our data to see the relative amount of both kinds of damage, for this we can use `df.groupby()` and `.count()` that we have seen several times above, you can pass multiple columns to `.groupby()`.

In [ ]:
df_img.groupby(['ORIGIN', 'QUAL']).count()

### Reshaping Dataframes

Dealing with tabular data, we often want to rotate, flatten or pivot data tables, Pandas allow us to do all this. 

In this tutorial, we will only look at:

- `df.pivot()`
- `pd.melt()`

But you can find full details if you refer to the [documentation](https://pandas.pydata.org/pandas-docs/stable/reshaping.html) - as always:


#### Pivot: `df.pivot()`

In [ ]:
df_img.head()

In [ ]:
df_img.pivot(index=None, columns='DIPTYPE', values='DIP').head()

In [ ]:
df_img.pivot(index=None, columns='DIPTYPE', values='DIP').mean()

#### Melt: `pd.melt()`

In [ ]:
df_img.head()

In [ ]:
pd.melt(df_img[df_img['ORIGIN'] == 'NATURAL'], id_vars=['DIPTYPE'], value_vars=['AZIM']).head()

### Handling multiple Dataframes

Joining, merging, concatenating tabular data is a large topic, by now you know: I always refer to the [docs](https://pandas.pydata.org/pandas-docs/stable/merging.html) and often check out [Chris Albon's page](https://chrisalbon.com/python/data_wrangling/pandas_join_merge_dataframe/).

In order to practise, let's load a new data set, we have four data frames representing:

1. GR upper hole section
2. GR lower hole section
3. Resistivity upper hole section
4. Resistivity lower hole section

We want to join these so we end up with a single DataFrame containing all data.

First import all four files:

In [ ]:
df_gr_upper = pd.read_csv('../data/df_gr_upper.csv', header=None, names=['GR'])
df_res_upper = pd.read_csv('../data/df_res_upper.csv', header=None, names=['RES'])
df_gr_lower = pd.read_csv('../data/df_gr_lower.csv', header=None, names=['GR'])
df_res_lower = pd.read_csv('../data/df_res_lower.csv', header=None, names=['RES'])

In [ ]:
df_gr_upper.head(2)

In [ ]:
df_gr_lower.head(2)

In [ ]:
df_res_upper.head(2)

In [ ]:
df_res_lower.head(2)

#### Making the GR and Res curves with `pd.concat()`

In [ ]:
df_gr = pd.concat([df_gr_upper, df_gr_lower])

In [ ]:
df_gr.tail()

In [ ]:
df_res = pd.concat([df_res_upper, df_res_lower])
df_res.tail()

#### Making a full log with GR and Res using `df.join()`

In [ ]:
df_log = df_gr.join(df_res, how='left')

In [ ]:
df_log.head()

In [ ]:
df_log.tail()

### Saving a modified DataFrame

There are many output formats you can get including:

- `df.to_csv()`
- `df.to_excel()`
- `df.to_html()`
- `df.to_json()`
- `df.to_feather()`
- `df.to_pickle()`

... and more

In [ ]:
df_img.to_csv('../cleaned_img_data.csv')

<hr />

<div>
<img src="https://avatars1.githubusercontent.com/u/1692321?s=50"><p style="text-align:center">© Agile Geoscience 2018</p>
</div>